# __고객 분석__
- __고객 분석(customer analysis)__이란 고객의 행동 데이터를 통해 시장을 세분화하고 예측 분석을 함으로써 가장 중요한 사업상 결정을 이끌어내는 과정을 말한다.
- __시장 세분화(market segmentation)__란 사용자 행동과 다른 유형의 공유 특성을 기반으로 사용자 기반 군집으로 나누는 과정을 말한다.
- 이는 각 __사용자 부문(user segment)__별로 맞춤형 제품을 제공하는 데 도움이 된다.
- 회사들은 다이렉트 마케팅(direct marketing), 부지 선정(site selection), 고객 확보(customer acquisition), 고객 관계 관리(customer relationship management, CRM)를 위해 시장 세분화 및 예측 모델에 의해 생성된 결과를 활용할 수 있다.

### index

- 연구문제 진술 소개
- 데이터셋 이해
- 고객 세분화를 위한 기준 접근법 구축 :
    - 기준 접근법 구현
    - 테스트 계량 이해
    - 기준 접근법의 결과 테스트
    - 기준 접근법의 문제점
    - 기준 접근법 최적화
- 고객 세분화를 위한 개선 접근법 구축 :
    - 개선 접근법 구현
    - 개선 접근법 테스트
    - 개선 접근법 문제점
    - 개선 접근법을 개선하는 방법 이해
- 고객 세분화를 위한 최선 접근법 :
    - 최선 접근법 구현
- 최선 접근법 테스트
- 다양한 분야에 대한 고객 세분화

# 1. 연구문제 소개
- 연구문제 :
    1. 고객의 구매 패턴에 따라 특정 부문에 고객을 범주화할 수 있다.
    2. 고객 세분화를 기준으로 향후에 어떤 제품을 구매할지를 예측할 수 있다.
- Input Data : 고객 4000명의 전자상거래 데이터(1년간의 구매 데이터)
- Output : 
    1. 고객 기반의 고객 부문 분류
    2. 고객의 첫번째 구매를 기준으로 금년과 내년의 구매를 예측

# 2. 데이터셋의 이해

- The UCI Machine Learning Repository에서 제공하는 Online Retail Dataset : 영국 소매 업체의 실제 거래 데이터
- 2010년 12월 1일부터 2011년 12월 9일까지의 실제 거래 데이터
- 사계절용 선물 판매에 대한 기록
- 소비자는 대체로 도매업체이다.
- 532,610개의 레코드

|변수명|설명|비고|
|:-----|:------|:------|
|1. InvoiceNo|송장번호|6자리 정수. 고유번호. c로 시작하면 취소를 나타낸다.|
|2. StockCode|재고번호|5자리 정수. 각 제품에 고유하게 지정된다.|
|3. Description|품목설명||
|4. Quantity|거래당 각 제품의 수량|숫자형식|
|5. InvoiceDate|송장 날짜와 시간|각 거래가 생성된 날짜와 시간|
|6. UnitPrice|단위 당 가격|영국 파운드화|
|7. CustomerID|고객 ID 번호|각 고객에게 할당된 고유5자리 번호|
|8. Country|고객의 지리적 정보|고객의 나라이름|

# 3. Baseline 구현

In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML

## 3-1) 데이터 준비

In [10]:
df_init = pd.read_csv("/home/pirl/Joshua/Machine_Learning_Practice_Projects/data/data.csv",
                      encoding="ISO-8859-1",
                     dtype={'CustomerID':str, 'InvoiceID':str})
print("DataFrame dimensions:", df_init.shape)
df_init['InvoiceDate'] = pd.to_datetime(df_init['InvoiceDate'])

DataFrame dimensions: (541909, 8)


In [12]:
df_init.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


## 3-2) EDA

### a. 결측치 제거

In [14]:
tab_info=pd.DataFrame(df_init.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_init.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_init.isnull().sum()/df_init.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
print('-' * 10 + " Display information about column types and number of null values " + '-' * 10 )
tab_info

---------- Display information about column types and number of null values ----------


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,1454,0,0,0,135080,0
null values (%),0,0,0.268311,0,0,0,24.9267,0


- 결측치 정보를 살펴보면 CustomerID 중 24%의 데이터가 CustomerID가 없는 것을 알 수 있다.
- 이는 이상 종료 데이터 입력에 해당한다.
- 기존의 CustomerID에 대응시킬 수 없기때문에 이 데이터들은 삭제한다.

In [15]:
df_init.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dataframe dimensions:', df_init.shape)
tab_info=pd.DataFrame(df_init.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_init.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_init.isnull().sum()/df_init.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
print('-' * 10 + " Display information about column types and number of null values " + '-' * 10 )
tab_info

Dataframe dimensions: (406829, 8)
---------- Display information about column types and number of null values ----------


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,0,0,0,0,0,0
null values (%),0,0,0,0,0,0,0,0


### b. 중복 데이터 입력 제거

In [18]:
# 중복되는 5225개의 데이터를 삭제
print('Duplicate data entries: {}'.format(df_init.duplicated().sum()))
df_init.drop_duplicates(inplace = True)

Duplicate data entries: 5225


### c. 다양한 데이터 특성을 위한 EDA

#### 1) 국가별

In [20]:
temp = df_init[['CustomerID', 'InvoiceNo','Country']].groupby(['CustomerID','InvoiceNo','Country']).count()
temp = temp.reset_index(drop=False)
countries = temp['Country'].value_counts()
print('No. of cuntries in dataframe: {}'.format(len(countries)))

No. of cuntries in dataframe: 37


In [21]:
temp_no_of_order_per_count = df_init[['CustomerID','Country']].groupby(['Country']).count()
temp_no_of_order_per_count = temp_no_of_order_per_count.reset_index(drop = False)

print('-' * 10 + " Contry-wise order calculation "+ '-' * 10)
print
print (temp_no_of_order_per_count.sort_values(
    by='CustomerID', ascending=False).rename(index=str,
                                        columns={"CustomerID": "Country wise number of order"}))

---------- Contry-wise order calculation ----------
                 Country  Country wise number of order
35        United Kingdom                        356728
14               Germany                          9480
13                France                          8475
10                  EIRE                          7475
30                 Spain                          2528
23           Netherlands                          2371
3                Belgium                          2069
32           Switzerland                          1877
26              Portugal                          1471
0              Australia                          1258
24                Norway                          1086
18                 Italy                           803
6        Channel Islands                           757
12               Finland                           695
7                 Cyprus                           611
31                Sweden                           461
1            

#### 2) 고객 및 제품 별

In [22]:
# value_counts()함수를 이용하여 사용자와 제품의 수를 파악한다.

pd.DataFrame([{'products': len(df_init['StockCode'].value_counts()),    
               'transactions': len(df_init['InvoiceNo'].value_counts()),
               'customers': len(df_init['CustomerID'].value_counts()),  
              }], columns = ['products', 'transactions', 'customers'], 
              index = ['quantity'])

,products,transactions,customers
quantity,3684,22190,4372


In [24]:
# 얼마나 많은 제품이 매 거래마다 구매되었는지 확인한다.
# InvoiceNo와 InvoiceDate 데이터 특성을 이용하여 모든 거래에서 구매한 제품 수를 계산한다.
temp = df_init.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
nb_products_per_basket[:10].sort_values('CustomerID')

,CustomerID,InvoiceNo,Number of products
0,12346,541431,1
1,12346,C541433,1
2,12347,537626,31
3,12347,542237,29
4,12347,549222,24
5,12347,556201,18
6,12347,562032,22
7,12347,573511,47
8,12347,581180,11
9,12348,539318,17


#### 3) 제품 범주 별

#### 4) 제품 범주 정의

## 3-3) 고객 범주 생성

## 3-4) 고객 분류